In [35]:
import requests

postulation ={
  "alias": "Felipe Terraza",
  "country": "CHL",
  "email": "felipe.terraza@sansano.usm.cl",
  "apply_role": "data"
}

url = 'https://makers-challenge.altscore.ai/v1/register'
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}


response = requests.post(url, json=postulation, headers=headers)

if response.status_code == 200:
    print('Listo')
else:
    print(f'error {response.status_code} {response.text}')

Listo


In [325]:
import time
from datetime import datetime

url = "https://makers-challenge.altscore.ai/v1/s1/e1/resources/measurement"
headers = {
    "accept": "application/json",
    "API-KEY": "2faf2ac84ff6440dad089735590d47b4" , # Reemplaza con tu clave real,
    "User-Agent": "Custom-Agent"

}

def mediciones(url,retries=10, delay=0.1):
    for attempt in range(retries):
        response = requests.get(url, headers=headers)
        data = response.json()
        if data['distance'] != 'failed to measure, try again':
            print(datetime.now())
            return data
        print(datetime.now())
        print(f"Intento {attempt + 1} fallido. Reintentando en {delay} segundos...")
        time.sleep(delay)
    raise Exception("Error: No se pudo obtener una medición válida después de varios intentos.")


for i in range(10):
    try:
        valor = mediciones(url)
        if valor['distance'] != 'failed to measure, try again':
            print("Datos de medición:", valor)
            break
    except Exception as e:
        print(datetime.now())
        print(e)



2024-12-25 17:42:03.313134
Intento 1 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:03.812443
Intento 2 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:04.285039
Intento 3 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:04.761710
Intento 4 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:05.222278
Intento 5 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:05.684429
Intento 6 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:06.163513
Intento 7 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:06.638000
Intento 8 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:07.119421
Intento 9 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:07.589788
Intento 10 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:07.692666
Error: No se pudo obtener una medición válida después de varios intentos.
2024-12-25 17:42:08.066075
Intento 1 fallido. Reintentando en 0.1 segundos...
2024-12-25 17:42:08.538857
Intento 2 fal

In [60]:
url_s1_e1 ='https://makers-challenge.altscore.ai/v1/s1/e1/solution'
headers_s1_e1 ={
    "accept" : "application/json",
    "API-KEY": "2faf2ac84ff6440dad089735590d47b4" ,
}
speed = {
    "speed": 405
}

response = requests.post(url_s1_e1, headers=headers_s1_e1, json=speed)
print(response.json())

{'result': 'correct'}


In [72]:
#S2_E2
import numpy as np
url = 'https://makers-challenge.altscore.ai/v1/s1/e2/resources/stars?page=(page)&sort-by=resonance&sort-direction=asc'
headers = {
    'accept': 'application/json',
    'API-KEY': '2faf2ac84ff6440dad089735590d47b4'
}
resonances = np.array([])
max = 0
page = 1
while max<735:
    response = requests.get(url.replace('(page)', str(page)), headers=headers)
    for star in response.json():
        resonances = np.append(resonances, star['resonance'])
    max = np.max(resonances)
    page += 1

mean = np.mean(resonances)



print(mean)


388.5


In [85]:
url_s2_e2 = 'https://makers-challenge.altscore.ai/v1/s1/e2/solution'

average_resonance = {
    "average_resonance" : 388
}

headers_s2_e2 = {
    "accept" : "application/json",
    "API-KEY": "2faf2ac84ff6440dad089735590d47b4" ,
}

response = requests.post(url_s2_e2, headers=headers_s2_e2, json=average_resonance)
print(response.json())


{'result': 'correct'}


In [137]:
import requests
import base64
import pandas as pd

r = requests.get('https://www.swapi.tech/api/people')
total_pages = r.json()['total_pages']
iterative_url = 'https://www.swapi.tech/api/people?page=(page)&limit=10'


info = {'names':[],'origin':[],'light':[],'dark':[]}
names = []
origin = []
for i in range(1,total_pages+1):
    response_people = requests.get(iterative_url.replace('(page)', str(i)))
    for person in response_people.json()['results']:
        names.append(person['name'])
        response_origin = requests.get(person['url']).json()
        origin_name = requests.get(response_origin['result']['properties']['homeworld']).json()['result']['properties']['name']
        origin.append(origin_name)

info['names'] = names
info['origin'] = origin

light = []
dark = []

header={ 
    "accept" : "application/json",
    "API-KEY": "2faf2ac84ff6440dad089735590d47b4" ,
}

for name in names:
    name_replace = name.replace(' ','%20')
    response = requests.get('https://makers-challenge.altscore.ai/v1/s1/e3/resources/oracle-rolodex?name='+name_replace,headers=header)
    decoded = base64.b64decode(response.json()['oracle_notes']).decode('utf-8')
    if 'belongs to the Light Side of the Force'.lower() in decoded.lower():
        light.append(1)
        dark.append(0)
    else:
        light.append(0)
        dark.append(1)

info['light'] = light
info['dark'] = dark

df_info = pd.DataFrame(info)
print(df_info)

planet_force = df_info.groupby('origin').agg({'light':'sum','dark':'sum'}).reset_index()
        

##solucion 

url_s1_e3 = 'https://makers-challenge.altscore.ai/v1/s1/e3/solution'

respuesta = {
    'planet' : planet_force[planet_force['light']==planet_force['dark']]['origin'].values[0]
}

solution = requests.post(url_s1_e3, json=respuesta,headers = header)

print(solution.json())


              names    origin  light  dark
0    Luke Skywalker  Tatooine      1     0
1             C-3PO  Tatooine      1     0
2             R2-D2     Naboo      1     0
3       Darth Vader  Tatooine      0     1
4       Leia Organa  Alderaan      1     0
..              ...       ...    ...   ...
77         Grievous     Kalee      0     1
78          Tarfful  Kashyyyk      1     0
79  Raymus Antilles  Alderaan      1     0
80        Sly Moore    Umbara      0     1
81       Tion Medon    Utapau      1     0

[82 rows x 4 columns]
{'result': 'correct'}


## Prueba 4

In [340]:
# Lista de posibles usernames
usernames = [
    "Elrond",
    "Galadriel",
    "Legolas",
    "Arwen",
    "Thranduil",
    "Erestor",
    "Glorfindel"
]

# Lista de posibles passwords
passwords = [
    "Lightbringer",
    "Shadowkeeper",
    "FadingLight",
    "HiddenMight",
    "AncientWar",
    "ElvenLore",
    "Starlight",
    "VeiledTruth"
]

url_s1_e4 = 'https://makers-challenge.altscore.ai/v1/s1/e4/solution'
for user in usernames:
    for password in passwords:
        print(user)
        print(password)
        login = {
            'username': user,
            'password': password
        }
        response = requests.post(url_s1_e4, json=login, headers=header)
        if response.status_code == 200:
            print(response.json())
        else:
            continue
        

Elrond
Lightbringer
{'result': 'incorrect'}
Elrond
Shadowkeeper
{'result': 'incorrect'}
Elrond
FadingLight
{'result': 'incorrect'}
Elrond
HiddenMight
{'result': 'incorrect'}
Elrond
AncientWar
{'result': 'incorrect'}
Elrond
ElvenLore
{'result': 'incorrect'}
Elrond
Starlight
{'result': 'incorrect'}
Elrond
VeiledTruth
{'result': 'incorrect'}
Galadriel
Lightbringer
{'result': 'incorrect'}
Galadriel
Shadowkeeper
{'result': 'incorrect'}
Galadriel
FadingLight
{'result': 'incorrect'}
Galadriel
HiddenMight
{'result': 'incorrect'}
Galadriel
AncientWar
{'result': 'incorrect'}
Galadriel
ElvenLore
{'result': 'incorrect'}
Galadriel
Starlight
{'result': 'incorrect'}
Galadriel
VeiledTruth
{'result': 'incorrect'}
Legolas
Lightbringer
{'result': 'incorrect'}
Legolas
Shadowkeeper
{'result': 'incorrect'}
Legolas
FadingLight
{'result': 'incorrect'}
Legolas
HiddenMight
{'result': 'incorrect'}
Legolas
AncientWar
{'result': 'incorrect'}
Legolas
ElvenLore
{'result': 'incorrect'}
Legolas
Starlight
{'result': 'i

In [ ]:
#DESAFIO 6

def pokemon_types():
    url = "https://pokeapi.co/api/v2/type/"
    response = requests.get(url)
    types = response.json()["results"]
    return types

def pokemons_by_type(type_url):
    url = type_url
    response = requests.get(url)
    pokemons = response.json()["pokemon"]
    return pokemons

def get_pokemon_height(pokemon_name):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_name}/"
    response = requests.get(url)
    data = response.json()
    return data["height"]


pokemons_by_type('https://pokeapi.co/api/v2/type/1/')


tipos_avg = {}
types = pokemon_types()
for type in range(1,19):
    height=[]
    for pokemon in pokemons_by_type('https://pokeapi.co/api/v2/type/'+str(type)+'/'):
        height.append(get_pokemon_height(pokemon['pokemon']['name']))
    tipos_avg[types[type-1]['name']] = round(sum(height)/(len(height)),3)

        
import json
json_tipos_avg = json.dumps(tipos_avg,indent=1)




value = {
 "bug": 19.529,
 "dark": 20.064,
 "dragon": 43.374,
 "electric": 16.491,
 "fairy": 19.41,
 "fighting": 22.6,
 "fire": 28.99,
 "flying": 16.597,
 "ghost": 14.728,
 "grass": 16.822,
 "ground": 19.323,
 "ice": 18.273,
 "normal": 15.665,
 "poison": 33.706,
 "psychic": 16.206,
 "rock": 18.02,
 "steel": 27.758,
 "water": 22.758
}
answer = {
    "heights" : value
}
header={ 
    "accept" : "application/json",
    "API-KEY": "2faf2ac84ff6440dad089735590d47b4" ,
}

url_s1_e6='https://makers-challenge.altscore.ai/v1/s1/e6/solution'
response = requests.post(url_s1_e6, json=answer,headers=header)

print(response.json())
        

In [257]:
import json
json_tipos_avg = json.dumps(tipos_avg)
print(json_tipos_avg)
json_tipos_avg = json_tipos_avg.replace ("'",'"')


{"bug": 19.529, "dark": 20.064, "dragon": 43.374, "electric": 16.491, "fairy": 19.41, "fighting": 22.6, "fire": 28.99, "flying": 16.597, "ghost": 14.728, "grass": 16.822, "ground": 19.323, "ice": 18.273, "normal": 15.665, "poison": 33.706, "psychic": 16.206, "rock": 18.02, "steel": 27.758, "water": 22.758}


## Prueba 8

In [326]:
import time
from datetime import datetime
header = {
    "accept" : "application/json",
    "API-KEY": "2faf2ac84ff6440dad089735590d47b4" ,
}




# Intentar abrir una puerta
def try_open_door(door_number, spell):
    response = requests.post('https://makers-challenge.altscore.ai/v1/s1/e8/actions/door',headers = header,json = {"encantamiento": spell, "numero": door_number})
    return response.json()

# Enviar la solución
# def send_solution(solution):
#     response = requests.post(solution_endpoint, json={"solution": solution})
#     return response.json()

# Interacción ejemplo
door_number = 1
spell = "revelio"  # Cambiar por pistas obtenidas
result = try_open_door(door_number, spell)
intento = 0
while intento < 3:
    if datetime.now().second == 0:
        door_number = 1
        result = try_open_door(door_number, spell)
        print(datetime.now().strftime("%H:%M:%S"))
        print(result)
        intento +=1
        time.sleep(0.7)
    
   



17:50:01
{'detail': 'Siempre es bueno un atajo, pero no en este caso.'}
17:51:00
{'response': '¡Bienvenido al desafío de Altwarts! Tienes suerte, aquí está la primera pista.'}
17:52:00
{'response': '¡Bienvenido al desafío de Altwarts! Tienes suerte, aquí está la primera pista.'}


In [336]:

import requests
import time
from datetime import datetime
# Define los endpoints
BASE_URL = "https://makers-challenge.altscore.ai/v1/s1/e8"
DOOR_ENDPOINT = f"{BASE_URL}/actions/door"
SOLUTION_ENDPOINT = f"{BASE_URL}/solution"

# Define la clave de la API
API_KEY = "2faf2ac84ff6440dad089735590d47b4"
HEADERS = {
    "accept": "application/json",
    "API-KEY": API_KEY
}

# Función para intentar abrir la puerta
def try_open_door(data=None):
    try:
        response = requests.post(DOOR_ENDPOINT, headers=HEADERS, json=data)
        print("Intentando abrir la puerta...")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.json()}")
        return response
    except Exception as e:
        print(f"Error al intentar abrir la puerta: {e}")
        return None

# Función para enviar la solución
def send_solution(solution):
    try:
        response = requests.post(SOLUTION_ENDPOINT, headers=HEADERS, json={"solution": solution})
        print("Enviando la solución...")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.json()}")
        return response
    except Exception as e:
        print(f"Error al enviar la solución: {e}")
        return None

# Resolución basada en los minutos exactos
def solve_puzzle():
    print("Esperando el momento exacto para intentar abrir la puerta...")
    while True:
        current_time = datetime.now()
        # Verificar si estamos en un minuto exacto (segundos = 0)
        if current_time.second == 0:
            print(f"Intentando abrir la puerta a las {current_time.strftime('%H:%M:%S')}")
            response = try_open_door()
            if response and response.status_code == 200:
                print("Puerta abierta con éxito en el minuto exacto!")
                print(response.json())
            elif response:
                print("Intento fallido. Respuesta obtenida:", response.json())
            else:
                print("Error desconocido al intentar abrir la puerta.")
            break
        time.sleep(0.5)  # Esperar medio segundo para evitar un bucle continuo

    # Segunda pista: Usar el hechizo "Revelio"
    print("Segunda pista: Probando con el hechizo 'Revelio'.")
    response = try_open_door({"action": "Revelio"})
    if response and response.status_code == 200:
        print("Respuesta al hechizo 'Revelio':", response.json())
        return(response.json())
    elif response:
        print("Intento fallido con 'Revelio'. Respuesta obtenida:", response.json())
    else:
        print("Error desconocido al intentar 'Revelio'.")

    # Probar otras acciones mágicas
    print("Intentando con otras palabras mágicas...")
    magic_words = ["Alohomora", "Lumos", "Nox", "Expelliarmus"]
    for word in magic_words:
        print(f"Probando con la palabra mágica: {word}")
        response = try_open_door({"action": word})
        if response and response.status_code == 200:
            print(f"Respuesta exitosa con la palabra mágica '{word}': {response.json()}")
            break
        elif response:
            print(f"Intento fallido con '{word}'. Respuesta obtenida: {response.json()}")
        else:
            print(f"Error desconocido al intentar con '{word}'.")
# Ejecutar la resolución del acertijo
solve_puzzle()


Esperando el momento exacto para intentar abrir la puerta...
Intentando abrir la puerta a las 18:14:00
Intentando abrir la puerta...
Status Code: 200
Response: {'response': '¡Bienvenido al desafío de Altwarts! Tienes suerte, aquí está la primera pista.'}
Puerta abierta con éxito en el minuto exacto!
{'response': '¡Bienvenido al desafío de Altwarts! Tienes suerte, aquí está la primera pista.'}
Segunda pista: Probando con el hechizo 'Revelio'.
Intentando abrir la puerta...
Status Code: 200
Response: {'response': '¡Bienvenido al desafío de Altwarts! Tienes suerte, aquí está la primera pista.'}
Respuesta al hechizo 'Revelio': {'response': '¡Bienvenido al desafío de Altwarts! Tienes suerte, aquí está la primera pista.'}


{'response': '¡Bienvenido al desafío de Altwarts! Tienes suerte, aquí está la primera pista.'}

# Desafio 9

In [341]:
!pip install h3

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/pyglet-1.5.20-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.3/801.3 kB 16.8 MB/s eta 0:00:00


In [344]:
# First, make sure to install the h3-py library:
# pip install h3

import h3

# Example coordinates
latitude = 37.7749 
longitude = -122.4194 
resolution = 9  # H3 resolution level (higher means smaller hexagons)

# Convert lat/lon to an H3 index
h3_index = h3.latlng_to_cell(latitude, longitude, resolution)

print(f"H3 Index: {h3_index}")

ModuleNotFoundError: No module named 'h3'